In [ ]:
import os
import pandas as pd
import pickle
from scipy.sparse import csr_matrix, save_npz
import numpy as np
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import joblib
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import cross_validate


In [ ]:
def load_data(raw_data_relative_path, filename):
    """
    Charge les données des fichiers CSV dans des DataFrames pandas.

    Args:
        raw_data_relative_path (str): Chemin vers le répertoire contenant les fichiers CSV.

    Returns:
        tuple: DataFrames pour les évaluations, les films et les liens.
    """
    try:
        if "ratings" in filename:
            df = pd.read_csv(
                f"{raw_data_relative_path}/{filename}",
                usecols=["userid", "movieid", "rating"],  # Sélectionner les colonnes
                dtype={"rating": "float32", "userid": str, "movieid": str},
            )
            return df
        elif "movies" in filename:
            df = pd.read_csv(
                f"{raw_data_relative_path}/{filename}",
                usecols=["movieid", "title", "genres"],  # Sélectionner les colonnes
                dtype={"movieid": str, "title": str, "genres": str},
            )
            return df
        print(f"Fichier {filename} chargé avec succès.")
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except pd.errors.EmptyDataError as e:
        print(f"No data: {e}")
    except Exception as e:
        print(f"An error occurred while loading data: {e}")


In [ ]:
def filterred_data(df):
    """
    Filtrer les données pour ne conserver que les films ayant reçu au moins 200 évaluations
    et les utilsateurs ayant évalués au moins 200 films.
    """
    user_counts = df["userid"].value_counts()
    users_with_more_than_200_ratings = user_counts[user_counts > 400].index

    # Étape 2 : Compter le nombre de notes par film
    movie_counts = df["movieid"].value_counts()
    movies_with_at_least_10_ratings = movie_counts[movie_counts >= 400].index

    # Étape 3 : Filtrer le DataFrame
    df = df[
        (df["userid"].isin(users_with_more_than_200_ratings))
        & (df["movieid"].isin(movies_with_at_least_10_ratings))
    ]

    return df

In [ ]:
def train_TFIDF_model(df, data_directory):
    """
    Entraîne un modèle TF-IDF pour extraire des caractéristiques des genres de films.
    """
    # Démarrer une nouvelle expérience MLflow
    # mlflow.start_run()

    start_time = datetime.now()  # Démarrer la mesure du temps

    # Vérifier les colonnes et le contenu
    print("Colonnes du DataFrame :", df.columns)
    print("Aperçu du DataFrame :")
    print(df.head())

    # Créer une instance de TfidfVectorizer
    tfidf = TfidfVectorizer()

    # Calculer la matrice TF-IDF
    tfidf_matrix = tfidf.fit_transform(df["genres"])

    # Afficher la taille de la matrice
    print(f"Dimensions de notre matrice TF-IDF : {tfidf_matrix.shape}")

    # Calculer la similarité cosinus par morceaux
    sim_cosinus = cosine_similarity(tfidf_matrix, tfidf_matrix)
    print(f"Dimensions de la matrice de similarité cosinus : {sim_cosinus.shape}")

    os.makedirs(data_directory, exist_ok=True)  # Crée le répertoire si nécessaire

    # Sauvegarder les éléments essentiels
    joblib.dump(tfidf, os.path.join(data_directory, 'tfidf_model.joblib'))
    joblib.dump(sim_cosinus, os.path.join(data_directory, 'sim_cosinus.joblib'))
    df[['movieid']].to_csv(os.path.join(data_directory, 'movieid.csv'), index=False)

    return tfidf, sim_cosinus, df['movieid']


In [ ]:
from tabulate import tabulate

def recommandations(titre, sim_cosinus, movieid, num_recommandations=10):
    """Fonction qui à partir des indices trouvés, renvoie les movie_id des films les plus similaires."""
    # récupérer dans idx l'indice associé au titre depuis la série indices
    #idx = indices[titre]
    idx = movieid.index(titre)
    # garder dans une liste les scores de similarité correspondants à l'index du film cible
    score_sim = list(enumerate(sim_cosinus[idx]))
    #  trier les scores de similarité, trouver les plus similaires et récupérer ses indices
    score_sim = sorted(score_sim, key=lambda x: x[1], reverse=True)
    # Obtenir les scores des 10 films les plus similaires
    top_similair = score_sim[1:num_recommandations+1]
    # Obtenir les indices des films
    res = [(movieid[idx], score) for idx, score in top_similair]
    # Renvoyer les movie_id des films les plus similaires
    return tabulate(res, headers=["movie_id", "Score de similarité"], tablefmt="pretty")

In [ ]:
def load_tfidf_model_artifacts(data_directory):
    """Charge les artefacts du modèle TF-IDF sauvegardés."""
    tfidf = joblib.load(os.path.join(data_directory, 'tfidf_model.joblib'))
    sim_cosinus = joblib.load(os.path.join(data_directory, 'sim_cosinus.joblib'))
    movieid = pd.read_csv(os.path.join(data_directory, 'movieid.csv'))['movieid'].tolist()
    return tfidf, sim_cosinus, movieid

In [ ]:
my_project_directory = os.path.join("/home/antoine/PROJET_MLOPS_RECO_MOVIES/")
raw_data_relative_path = os.path.join(my_project_directory, "data/raw/silver")
movies = load_data(raw_data_relative_path, "processed_movies.csv")
ratings = load_data(raw_data_relative_path, "processed_ratings.csv")
df = pd.merge(ratings, movies, on="movieid", how="left")
print(df.shape)
# df = filterred_data(df)
print(df.shape)
df['userid'] = df['userid'].astype('category')
df['title'] = df['title'].astype('category')
df.head()

(20000263, 5)
(20000263, 5)


,userid,movieid,rating,title,genres
0,1,2,3.5,Jumanji,"Adventure, Children, Fantasy"
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...","Adventure, Drama, Fantasy, Mystery, Sci-Fi"
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),"Mystery, Sci-Fi, Thriller"
3,1,47,3.5,Seven (a.k.a. Se7en),"Mystery, Thriller"
4,1,50,3.5,"Usual Suspects, The","Crime, Mystery, Thriller"


In [5]:
def limit_ratings_per_user(df, n=5):
  """
  Limite le DataFrame à un maximum de n notations par utilisateur.

  Args:
    df: Le DataFrame contenant les colonnes 'userid' et 'rating'.
    n: Le nombre maximum de notations à conserver par utilisateur.

  Returns:
    Un nouveau DataFrame contenant au plus n notations par utilisateur.
  """
  limited_df = df.groupby('userid').head(n)
  print(df.shape, limited_df.shape)
  return limited_df


limited_df = limit_ratings_per_user(df, n=2)
limited_df.head(10)

(20000263, 5) (276986, 5)


,userid,movieid,rating,title,genres
0,1,2,3.5,Jumanji,"Adventure, Children, Fantasy"
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...","Adventure, Drama, Fantasy, Mystery, Sci-Fi"
175,2,3,4.0,Grumpier Old Men,"Comedy, Romance"
176,2,62,5.0,Mr. Holland's Opus,Drama
236,3,1,4.0,Toy Story,"Adventure, Animation, Children, Comedy, Fantasy"
237,3,24,3.0,Powder,"Drama, Sci-Fi"
423,4,6,3.0,Heat,"Action, Crime, Thriller"
424,4,10,4.0,GoldenEye,"Action, Adventure, Thriller"
451,5,2,3.0,Jumanji,"Adventure, Children, Fantasy"
452,5,11,5.0,"American President, The","Comedy, Drama, Romance"


In [7]:
import dask.dataframe as dd
import dask.array as da

def train_matrix_factorization_model_dask(df, data_directory, npartitions=4, n_components=10):
    """
    Entraîne un modèle de factorisation matricielle avec Dask pour prédire les évaluations des utilisateurs.
    """

    start_time = datetime.now()

    # 1. Convertir le DataFrame pandas en DataFrame Dask
    ddf = dd.from_pandas(df, npartitions=npartitions)

    # 2. Calcul de la table pivot avec Dask
    mat_ratings = ddf.pivot_table(values='rating', columns='title', index='userid').compute() # .compute() ici

    mat_ratings = mat_ratings.fillna(0)
    # 3. Conversion en matrice creuse
    sparse_ratings = csr_matrix(mat_ratings)

    user_ids = mat_ratings.index.tolist()
    titles = mat_ratings.columns.tolist()

    # 4. Factorisation matricielle
    svd = TruncatedSVD(n_components=n_components)
    ratings_red = svd.fit_transform(sparse_ratings.T)

    # 5. Calcul de la similarité cosinus
    item_similarity = cosine_similarity(ratings_red)
    item_similarity = pd.DataFrame(item_similarity, index=titles, columns=titles)

    # 6. Sauvegarder les éléments essentiels
    joblib.dump(svd, os.path.join(data_directory, 'svd_model.joblib'))
    with open(os.path.join(data_directory, 'titles.pkl'), 'wb') as f:
        pickle.dump(titles, f)
    joblib.dump(item_similarity, os.path.join(data_directory, 'item_similarity.joblib'))
    joblib.dump(mat_ratings, os.path.join(data_directory, 'mat_ratings.joblib'))

    return mat_ratings, item_similarity, titles, svd

In [6]:
from memory_profiler import profile

@profile
def train_matrix_factorization_model(df, data_directory):
    """
    Entraîne un modèle de factorisation matricielle pour prédire les évaluations des utilisateurs.
    """
    # Démarrer une nouvelle expérience MLflow
    # mlflow.start_run()

    start_time = datetime.now()  # Démarrer la mesure du temps

    # df = df.sample(frac=0.5, random_state=42).reset_index(drop=True)
    mat_ratings = pd.pivot_table(
        data=df, values="rating", columns="title", index="userid"
    )
    mat_ratings = (
        mat_ratings + 1
    )  # On ajoute 1 à toutes les notes pour éviter les problèmes de division par 0
    mat_ratings = mat_ratings.fillna(0)
    sparse_ratings = csr_matrix(mat_ratings)
    user_ids = mat_ratings.index.tolist()
    titles = mat_ratings.columns.tolist()
    # Appliquer la factorisation matricielle
    svd = TruncatedSVD(n_components=10)
    ratings_red = svd.fit_transform(sparse_ratings.T)
    item_similarity = cosine_similarity(ratings_red)
    item_similarity = pd.DataFrame(item_similarity, index=titles, columns=titles)

    # Sauvegarder les éléments essentiels
    joblib.dump(svd, os.path.join(data_directory, 'svd_model.joblib'))
    with open(os.path.join(data_directory, 'titles.pkl'), 'wb') as f:
        pickle.dump(titles, f)
    joblib.dump(item_similarity, os.path.join(data_directory, 'item_similarity.joblib'))
    joblib.dump(mat_ratings, os.path.join(data_directory, 'mat_ratings.joblib'))

    return mat_ratings, item_similarity, titles, svd

In [7]:
mat_ratings, item_similarity, titles, svd = train_matrix_factorization_model(limited_df, os.path.join(my_project_directory, "data/models/"))

ERROR: Could not find file /tmp/ipykernel_84336/3653479148.py


: 

In [12]:
def pred_item(mat_ratings, item_similarity, k, user_id):
    # Sélectionner dans mat_ratings les films qui n'ont pas été encore lu par le user
    to_predict = mat_ratings.loc[user_id][mat_ratings.loc[user_id]==0]
    # Itérer sur tous ces films
    for i in to_predict.index:
        #Trouver les k films les plus similaires en excluant le film lui-même
        similar_items = item_similarity.loc[i].sort_values(ascending=False)[1:k+1]
        # Calcul de la norme du vecteur similar_items
        norm = np.sum(np.abs(similar_items))
        # Récupérer les notes données par l'utilisateur aux k plus proches voisins
        ratings = mat_ratings[similar_items.index].loc[user_id]
        # Calculer le produit scalaire entre ratings et similar_items
        scalar_prod = np.dot(ratings,similar_items)
        #Calculer la note prédite pour le film i
        pred = scalar_prod / norm
        # Remplacer par la prédiction
        to_predict[i] = pred
    return to_predict

In [16]:
recommandation = pred_item(mat_ratings, item_similarity, 12, '10001')
recommandation = recommandation.sort_values(ascending=False).head(12)
titles = recommandation.index.tolist()
print(titles)

['Snatch', 'Professional, The (Le professionnel)', 'Fight Club', 'Graduate, The', 'Lost in Translation', "Monty Python's Life of Brian", 'Maltese Falcon, The', 'Birds, The', 'Super Size Me', 'Run Lola Run (Lola rennt)', '2001: A Space Odyssey', 'Back to the Future']


In [15]:
print(mat_ratings)

title   'Til There Was You  'burbs, The  (500) Days of Summer  \
userid                                                          
100009                 0.0          0.0                   0.0   
10001                  0.0          0.0                   0.0   
100020                 0.0          0.0                   0.0   
100023                 0.0          0.0                   0.0   
100052                 0.0          0.0                   0.0   
...                    ...          ...                   ...   
99941                  0.0          0.0                   0.0   
99967                  0.0          0.0                   0.0   
99968                  0.0          0.0                   0.0   
99985                  0.0          0.0                   0.0   
99991                  0.0          0.0                   0.0   

title   *batteries not included  ...And Justice for All   10  \
userid                                                         
100009                    

In [16]:
title_to_movieid = dict(zip(movies["title"], movies["movieid"]))


In [17]:
reco = [title_to_movieid[title] for title in titles]
print(reco)

['33823', '52', '4738', '3566', '1407', '5128', '1381', '4487', '2558', '1353', '3269', '2082']


In [1]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd
from surprise.model_selection import cross_validate
import os
import pickle

In [2]:
def load_model(filepath: str, version: str):
    """Charge le modèle entraîné depuis un fichier."""
    base, ext = os.path.splitext(filepath)
    versioned_filepath = f"{base}_{version}{ext}"

    with open(versioned_filepath, 'rb') as file:
        model_data = pickle.load(file)
        model = model_data['model']
        reader = model_data['reader']
        print(f'Modèle chargé depuis {versioned_filepath}')
        return model, reader

In [4]:
model, reader = load_model(os.path.join("/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/models/svd_model.pkl"), 'v1')

Modèle chargé depuis /home/antoine/PROJET_MLOPS_RECO_MOVIES/data/models/svd_model_v1.pkl


In [5]:
import pandas as pd
from surprise import SVD, Dataset

def recommend_movies(model: SVD, user_id: int, df: pd.DataFrame, top_n: int = 10) -> list:
    """
    Recommande des films à un utilisateur en fonction de ses évaluations prédites.

    :param model: Le modèle SVD entraîné.
    :param user_id: L'ID de l'utilisateur pour lequel on veut des recommandations.
    :param df: DataFrame contenant les données des films et les évaluations.
    :param top_n: Le nombre de films à recommander.
    :return: Une liste des top_n films recommandés.
    """
    # 1. Obtenir la liste des films que l'utilisateur a déjà évalués
    movies_already_rated = df[df['userid'] == user_id]['movieid'].unique()

    # 2. Créer une liste de tous les films possibles
    all_movie_ids = df['movieid'].unique()

    # 3. Filtrer les films que l'utilisateur n'a pas encore évalués
    movies_to_predict = [movie_id for movie_id in all_movie_ids if movie_id not in movies_already_rated]

    # 4. Faire des prédictions pour chaque film non évalué
    predictions = []
    for movie_id in movies_to_predict:
        predictions.append((movie_id, model.predict(user_id, movie_id).est))

    # 5. Trier les prédictions par ordre décroissant d'évaluation prédite
    predictions.sort(key=lambda x: x[1], reverse=True)

    # 6. Retourner les top_n films
    top_recommendations = [movie_id for movie_id, _ in predictions[:top_n]]
    return top_recommendations

In [6]:
movies = pd.read_csv('/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/raw/silver/processed_movies.csv')
movie_titles = dict(zip(movies["movieid"], movies["title"]))
ratings = pd.read_csv('/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/raw/silver/processed_ratings.csv')
df = pd.merge(ratings, movies, on="movieid", how="left")

In [10]:
recommand_movies = recommend_movies(model, 3, df, 10)

In [11]:
print(recommand_movies)

[114635, 88570, 26109, 78064, 127021, 124273, 5952, 80337, 45691, 44421]


In [12]:
for i in recommand_movies:
    print(movie_titles[i])

Look of Silence, The
Welfare
Crooks in Clover (a.k.a. Monsieur Gangster) (Les tontons flingueurs)
Ween Live in Chicago
Rewind This!
Kevin Smith: Too Fat For 40
Lord of the Rings: The Two Towers, The
Still Bill
Edvard Munch
Personal Journey with Martin Scorsese Through American Movies, A


AttributeError: 'builtin_function_or_method' object has no attribute 'tolist'

In [14]:
title = [movie_titles[movied] for movied in recommand_movies]
print(title)

['Look of Silence, The', 'Welfare', 'Crooks in Clover (a.k.a. Monsieur Gangster) (Les tontons flingueurs)', 'Ween Live in Chicago', 'Rewind This!', 'Kevin Smith: Too Fat For 40', 'Lord of the Rings: The Two Towers, The', 'Still Bill', 'Edvard Munch', 'Personal Journey with Martin Scorsese Through American Movies, A']


In [6]:
# Charger le fichier avec scipy.sparse.load_npz
from scipy.sparse import load_npz
cos_sim = load_npz('/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/models/cosine_similarity_tfidf.npz')


In [8]:
print('RECOMMNDATION SIMILARITES COSINUS')
print(recommandations('Toy Story', cos_sim))

RECOMMNDATION SIMILARITES COSINUS


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed